In [166]:
import pandas as pd
import numpy as np

#load the data
filepath = '../Data/Data.txt'
df = pd.read_csv(filepath, delimiter=';')

In [143]:
df.head(10)

,HourUTC,HourDK,PriceArea,ConsumerType_DE35,TotalCon
0,2023-06-30 21:00,2023-06-30 23:00,DK1,111,65920
1,2023-06-30 21:00,2023-06-30 23:00,DK2,111,47018
2,2023-06-30 21:00,2023-06-30 23:00,DK1,112,1313
3,2023-06-30 21:00,2023-06-30 23:00,DK2,112,933
4,2023-06-30 21:00,2023-06-30 23:00,DK1,119,21806
5,2023-06-30 21:00,2023-06-30 23:00,DK2,119,26458
6,2023-06-30 21:00,2023-06-30 23:00,DK1,121,222108
7,2023-06-30 21:00,2023-06-30 23:00,DK2,121,113923
8,2023-06-30 21:00,2023-06-30 23:00,DK1,122,25068
9,2023-06-30 21:00,2023-06-30 23:00,DK2,122,24129


In [167]:
# Check Totalcon have 0 values and replace them with mean

df['TotalCon'] = df['TotalCon'].replace(0, np.nan)

# Show the number with NaN values
missing_values = df.isnull().sum()
print(missing_values)

HourUTC              0
HourDK               0
PriceArea            0
ConsumerType_DE35    0
TotalCon             0
dtype: int64


In [168]:
# Fill NaN values with the mean of 'TotalCon'
df['TotalCon'] = df['TotalCon'].fillna(df['TotalCon'].mean())

In [169]:
df['EventTimestamp'] = pd.to_datetime(df['HourDK'])
df['idx'] = pd.to_datetime(df.EventTimestamp)
df['DMY'] = df['idx'].dt.strftime('%d-%m-%Y')
df= df.set_index('idx')
df['Hour'] = df.index.hour
df['Minute'] = df.index.minute
df['DayOfWeek'] = df.index.dayofweek
df['Quarter'] = df.index.quarter
df['Month'] = df.index.month
df['Day'] = df.index.day
df['Year'] = df.index.year
df['Season'] = df['Month'] % 12 // 3 + 1
df['Dayofyear'] = df.index.dayofyear
df['DayOfMonth'] = df.index.day
df['WeekOfYear'] = df.index.isocalendar().week

# Additional features
df['IsWeekend'] = df['DayOfWeek'].isin([5, 6])
df['IsMonthStart'] = (df['DayOfMonth'] == 1)
df['IsMonthEnd'] = (df['DayOfMonth'] == df.index.days_in_month)
df['IsQuarterStart'] = (df['DayOfMonth'] == 1) & (df['Month'] % 3 == 1)
df['IsQuarterEnd'] = (df['DayOfMonth'] == df.groupby(['Year', 'Quarter'])['DayOfMonth'].transform('max'))

# Additional features
df['IsWorkingDay'] = df['DayOfWeek'].isin([0, 1, 2, 3, 4])
df['IsBusinessHours'] = df['Hour'].between(9, 17)
df['IsPeakHour'] = df['Hour'].isin([8, 12, 18])

# Minute-level features
df['MinuteOfDay'] = df['Hour'] * 60 + df['Minute']
df['MinuteOfWeek'] = (df['DayOfWeek'] * 24 * 60) + df['MinuteOfDay']

In [170]:
def check_data_complete(df):
    # Group by ConsumerType_DE35 and PriceArea, and count unique hours
    group_counts = df.groupby(['ConsumerType_DE35', 'PriceArea', 'DMY']).size()
    print (group_counts[group_counts != 24])
    # Check which groups have less than 24 unique hours
    incomplete_data = group_counts[group_counts != 24].index.tolist()
    print(f"ConsumerType_DE35 and PriceArea groups with incomplete data: {incomplete_data};")
    return incomplete_data



In [171]:
incomplete_combinations = check_data_complete(df)

ConsumerType_DE35  PriceArea  DMY       
111                DK1        25-10-2020    25
                              26-03-2023    23
                              27-03-2022    23
                              28-03-2021    23
                              30-10-2022    25
                                            ..
999                DK2        26-03-2023    23
                              27-03-2022    23
                              28-03-2021    23
                              30-10-2022    25
                              31-10-2021    25
Length: 456, dtype: int64
ConsumerType_DE35 and PriceArea groups with incomplete data: [(111, 'DK1', '25-10-2020'), (111, 'DK1', '26-03-2023'), (111, 'DK1', '27-03-2022'), (111, 'DK1', '28-03-2021'), (111, 'DK1', '30-10-2022'), (111, 'DK1', '31-10-2021'), (111, 'DK2', '25-10-2020'), (111, 'DK2', '26-03-2023'), (111, 'DK2', '27-03-2022'), (111, 'DK2', '28-03-2021'), (111, 'DK2', '30-10-2022'), (111, 'DK2', '31-10-2021'), (112, 'DK1', '25-1

### Filter unusual data during the day

In [172]:
df_filtered = df[df[['ConsumerType_DE35', 'PriceArea', 'DMY']].apply(tuple, 1).isin(incomplete_combinations)]

In [173]:
df_filtered.shape

(10944, 28)

In [174]:
# Get all data is duplicated base on 3 columns ConsumerType_DE35, PriceArea, and HourDK
# Group by the three columns and filter out groups with count > 1
df_duplicated = df_filtered.groupby(['ConsumerType_DE35', 'PriceArea', 'HourDK']).filter(lambda x: len(x) > 1)


#df_duplicated = df_duplicated[df_duplicated.duplicated(subset=['ConsumerType_DE35', 'PriceArea', 'HourDK'], keep=False)]


In [175]:
# Export Duplicate data to CSV
df_duplicated.to_csv('../Data/DuplicatedData.csv', index=False)

In [161]:
# Remove duplicated data from the original data keeping the first occurence

#df = df.drop_duplicates(subset=['ConsumerType_DE35', 'PriceArea', 'HourDK'], keep='first')

In [176]:
# Step 1: Identify duplicates and calculate means

mean_totalcon = df_duplicated.groupby(['ConsumerType_DE35', 'PriceArea', 'HourDK'])['TotalCon'].transform('mean')



In [177]:
df_duplicated

,HourUTC,HourDK,PriceArea,ConsumerType_DE35,TotalCon,EventTimestamp,DMY,Hour,Minute,DayOfWeek,...,IsWeekend,IsMonthStart,IsMonthEnd,IsQuarterStart,IsQuarterEnd,IsWorkingDay,IsBusinessHours,IsPeakHour,MinuteOfDay,MinuteOfWeek
idx,,,,,,,,,,,,,,,,,,,,,
2022-10-30 02:00:00,2022-10-30 01:00,2022-10-30 02:00,DK1,111,45875,2022-10-30 02:00:00,30-10-2022,2,0,6,...,True,False,False,False,False,False,False,False,120,8760
2022-10-30 02:00:00,2022-10-30 01:00,2022-10-30 02:00,DK1,112,1299,2022-10-30 02:00:00,30-10-2022,2,0,6,...,True,False,False,False,False,False,False,False,120,8760
2022-10-30 02:00:00,2022-10-30 01:00,2022-10-30 02:00,DK1,119,18153,2022-10-30 02:00:00,30-10-2022,2,0,6,...,True,False,False,False,False,False,False,False,120,8760
2022-10-30 02:00:00,2022-10-30 01:00,2022-10-30 02:00,DK1,121,152261,2022-10-30 02:00:00,30-10-2022,2,0,6,...,True,False,False,False,False,False,False,False,120,8760
2022-10-30 02:00:00,2022-10-30 01:00,2022-10-30 02:00,DK1,122,26038,2022-10-30 02:00:00,30-10-2022,2,0,6,...,True,False,False,False,False,False,False,False,120,8760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-10-25 02:00:00,2020-10-25 00:00,2020-10-25 02:00,DK2,447,19368,2020-10-25 02:00:00,25-10-2020,2,0,6,...,True,False,False,False,False,False,False,False,120,8760
2020-10-25 02:00:00,2020-10-25 00:00,2020-10-25 02:00,DK2,450,7860,2020-10-25 02:00:00,25-10-2020,2,0,6,...,True,False,False,False,False,False,False,False,120,8760
2020-10-25 02:00:00,2020-10-25 00:00,2020-10-25 02:00,DK2,461,14473,2020-10-25 02:00:00,25-10-2020,2,0,6,...,True,False,False,False,False,False,False,False,120,8760


In [178]:
df_duplicated['Mean'] = mean_totalcon

In [179]:
df_duplicated 

,HourUTC,HourDK,PriceArea,ConsumerType_DE35,TotalCon,EventTimestamp,DMY,Hour,Minute,DayOfWeek,...,IsMonthStart,IsMonthEnd,IsQuarterStart,IsQuarterEnd,IsWorkingDay,IsBusinessHours,IsPeakHour,MinuteOfDay,MinuteOfWeek,Mean
idx,,,,,,,,,,,,,,,,,,,,,
2022-10-30 02:00:00,2022-10-30 01:00,2022-10-30 02:00,DK1,111,45875,2022-10-30 02:00:00,30-10-2022,2,0,6,...,False,False,False,False,False,False,False,120,8760,47326.5
2022-10-30 02:00:00,2022-10-30 01:00,2022-10-30 02:00,DK1,112,1299,2022-10-30 02:00:00,30-10-2022,2,0,6,...,False,False,False,False,False,False,False,120,8760,1306.0
2022-10-30 02:00:00,2022-10-30 01:00,2022-10-30 02:00,DK1,119,18153,2022-10-30 02:00:00,30-10-2022,2,0,6,...,False,False,False,False,False,False,False,120,8760,18445.0
2022-10-30 02:00:00,2022-10-30 01:00,2022-10-30 02:00,DK1,121,152261,2022-10-30 02:00:00,30-10-2022,2,0,6,...,False,False,False,False,False,False,False,120,8760,159257.0
2022-10-30 02:00:00,2022-10-30 01:00,2022-10-30 02:00,DK1,122,26038,2022-10-30 02:00:00,30-10-2022,2,0,6,...,False,False,False,False,False,False,False,120,8760,26604.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-10-25 02:00:00,2020-10-25 00:00,2020-10-25 02:00,DK2,447,19368,2020-10-25 02:00:00,25-10-2020,2,0,6,...,False,False,False,False,False,False,False,120,8760,18910.5
2020-10-25 02:00:00,2020-10-25 00:00,2020-10-25 02:00,DK2,450,7860,2020-10-25 02:00:00,25-10-2020,2,0,6,...,False,False,False,False,False,False,False,120,8760,7564.0
2020-10-25 02:00:00,2020-10-25 00:00,2020-10-25 02:00,DK2,461,14473,2020-10-25 02:00:00,25-10-2020,2,0,6,...,False,False,False,False,False,False,False,120,8760,14724.0


In [136]:
incomplete_combinations = check_data_complete(df)

ConsumerType_DE35  PriceArea  DMY       
111                DK1        26-03-2023    23
                              27-03-2022    23
                              28-03-2021    23
                   DK2        26-03-2023    23
                              27-03-2022    23
                                            ..
999                DK1        27-03-2022    23
                              28-03-2021    23
                   DK2        26-03-2023    23
                              27-03-2022    23
                              28-03-2021    23
Length: 228, dtype: int64
ConsumerType_DE35 and PriceArea groups with incomplete data: [(111, 'DK1', '26-03-2023'), (111, 'DK1', '27-03-2022'), (111, 'DK1', '28-03-2021'), (111, 'DK2', '26-03-2023'), (111, 'DK2', '27-03-2022'), (111, 'DK2', '28-03-2021'), (112, 'DK1', '26-03-2023'), (112, 'DK1', '27-03-2022'), (112, 'DK1', '28-03-2021'), (112, 'DK2', '26-03-2023'), (112, 'DK2', '27-03-2022'), (112, 'DK2', '28-03-2021'), (119, 'DK1', '26-0

In [195]:
# Assuming df_duplicated is already defined as the DataFrame subset with duplicated rows
mean_totalcon = df_duplicated.groupby(['ConsumerType_DE35', 'PriceArea', 'HourDK'])['TotalCon'].mean()

# Update TotalCon in df using mean values from df_duplicated
df['TotalCon'] = df.apply(lambda row: row['TotalCon'] if pd.notna(row['TotalCon']) else mean_totalcon.loc[(row['ConsumerType_DE35'], row['PriceArea'], row['HourDK'])], axis=1)

# Print the updated DataFrame to verify
print(df)


                              HourUTC            HourDK PriceArea  \
idx                                                                 
2023-06-30 23:00:00  2023-06-30 21:00  2023-06-30 23:00       DK1   
2023-06-30 23:00:00  2023-06-30 21:00  2023-06-30 23:00       DK2   
2023-06-30 23:00:00  2023-06-30 21:00  2023-06-30 23:00       DK1   
2023-06-30 23:00:00  2023-06-30 21:00  2023-06-30 23:00       DK2   
2023-06-30 23:00:00  2023-06-30 21:00  2023-06-30 23:00       DK1   
...                               ...               ...       ...   
2020-07-01 00:00:00  2020-06-30 22:00  2020-07-01 00:00       DK2   
2020-07-01 00:00:00  2020-06-30 22:00  2020-07-01 00:00       DK2   
2020-07-01 00:00:00  2020-06-30 22:00  2020-07-01 00:00       DK2   
2020-07-01 00:00:00  2020-06-30 22:00  2020-07-01 00:00       DK2   
2020-07-01 00:00:00  2020-06-30 22:00  2020-07-01 00:00       DK2   

                     ConsumerType_DE35  TotalCon      EventTimestamp  \
idx                           

In [196]:
df = df.drop_duplicates(subset=['ConsumerType_DE35', 'PriceArea', 'HourDK'], keep='first')

In [197]:
incomplete_combinations = check_data_complete(df)

ConsumerType_DE35  PriceArea  DMY       
111                DK1        26-03-2023    23
                              27-03-2022    23
                              28-03-2021    23
                   DK2        26-03-2023    23
                              27-03-2022    23
                                            ..
999                DK1        27-03-2022    23
                              28-03-2021    23
                   DK2        26-03-2023    23
                              27-03-2022    23
                              28-03-2021    23
Length: 228, dtype: int64
ConsumerType_DE35 and PriceArea groups with incomplete data: [(111, 'DK1', '26-03-2023'), (111, 'DK1', '27-03-2022'), (111, 'DK1', '28-03-2021'), (111, 'DK2', '26-03-2023'), (111, 'DK2', '27-03-2022'), (111, 'DK2', '28-03-2021'), (112, 'DK1', '26-03-2023'), (112, 'DK1', '27-03-2022'), (112, 'DK1', '28-03-2021'), (112, 'DK2', '26-03-2023'), (112, 'DK2', '27-03-2022'), (112, 'DK2', '28-03-2021'), (119, 'DK1', '26-0